In [2]:
#initials
import requests
from bs4 import BeautifulSoup
baseUrl = 'https://orientacion.universia.net.co/'

In [4]:
#initial info scrapper 
url = baseUrl + "/buscar5/pregrados-en-medellin/1/26.html"
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
soup.prettify()
print("")

In [5]:
#info extraction for every carreer
pages = []
for a in soup.findAll("article"):
    link=a.find('a')
    r2 = requests.get(baseUrl+str(link['href']))
    soup2 = BeautifulSoup(r2.content, 'html.parser') 
    pages.append(soup2.prettify())   

In [8]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
baseUrl = 'https://orientacion.universia.net.co/'

def get_search_results():
    url = baseUrl + "/buscar5/pregrados-en-medellin/1/26.html"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    return soup.prettify()

def get_all_carreers():
    soup = get_search_results()
    pages = []
    for a in soup.findAll("article"):
        link=a.find('a')
        r = requests.get(baseUrl+str(link['href']))
        page = BeautifulSoup(r.content, 'html.parser') 
        pages.append(page.prettify()) 
    return pages

def file_print(text):
    f = open("file2.txt", "w", encoding='utf-8')
    f.write(text)
    f.close()

def get_data_from_file():
    f = open("file.txt", "r", encoding='utf-8')
    return f.read().split("xd"*5)[:-1]




def get_table(soup):
    soup = BeautifulSoup(soup, 'html.parser')
    table = soup.find("table", {"class": "TablaSnies"})
    rows = table.findAll('tr')
    data = [[td.findChildren(text=True) for td in tr.findAll("td")] for tr in rows]
    return data

def get_tables(careers):
    careers_data = []

    for career in careers:
        career_data = {}
        career = get_table(career)
        for attr in career:

            try:
                column = attr[0][1].strip(" \n ")[:-1]
                value =  attr[1][0].strip(" \n ")
                career_data[column] = value
            except:
                continue
        careers_data.append(career_data)
    return careers_data
    
careers = get_data_from_file()
df = pd.DataFrame.from_dict(get_tables(careers))
df


,SNIES,Registro calificado,Nivel de formación,Tipo de formación,Título otorgado,Modalidad,Duración,Créditos
0,101342,"Resolución 10628 del 40869, vigencia 7 años",Posgrado,Doctorado,Doctor En Ciencias Sociales,Presencial,8 Semestres,102
1,107303,"Resolución 14036 del 15 de agosto de 2018, Vig...",Posgrado,Maestría,Magíster en Ciencias de los Datos y la Analítica,Presencial,4 semestres,NaN
2,54693,"Resolución 3347 del 41712, vigencia 7 años",Posgrado,Especialización Universitaria,Especialista En Gestión E Intervención Del Pat...,Presencial,2 Semestres,26
3,103135,"Resolución 3343 del 41712, vigencia 7 años",Posgrado,Especialización Universitaria,Especialista En Legislación Financiera Y Del M...,Presencial,2 Semestres,26
4,1249,"Resolución 04422 del 08 de abril de 2015, vige...",Pregrado,Universitaria,Ingeniero de Producción,Presencial,10 Semestres,178
...,...,...,...,...,...,...,...,...
348,103604,Resolución 13045 del 13 de agosto del 2014. Vi...,Posgrado,Doctorado,Doctor en Ingeniería Matemática,Presencial,8 Semestres,92
349,103166,Resolución 3891 de marzo 20 de 2014 por 7 años,Posgrado,Maestría,Magíster en tecnologías de la información y la...,Presencial,2 Años,52
350,12037,Resolución 2882 de marzo 6 de 2015 por 7 años,Posgrado,Maestría,Magister En Epidemiologia,Presencial,4 Semestres,56
351,1203,"Resolución 1246 del 40595, vigencia 8 años",Pregrado,Universitaria,Ingeniero Quimico,Presencial,10 Semestres,160
